<a href="https://colab.research.google.com/github/CER-REC/esa-data-bank_banque-donnees-ees/blob/master/Codes/Section_05_VEC_Labeling_for_Tables/02_labeling_VCs_with_zero_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install tqdm

     |████████████████████████████████| 2.5MB 15.0MB/s 
     |████████████████████████████████| 901kB 38.5MB/s 
     |████████████████████████████████| 3.3MB 49.5MB/s 


In [2]:
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from tqdm import tqdm

nltk.download('stopwords', quiet=True)

True

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
df_joined = pd.read_csv('/content/drive/MyDrive/data/esa_index_files/zero_shot_esa_index_for_labeling_128_max_tokens.csv', index_col=0)

In [5]:
len(df_joined)

1000

In [6]:
df_joined.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,full_text
8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2S0,2758825,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, All",GH-002-2015,TERA a CHTM HILL Company,Operating,GC-126,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,Section 8: Cumulative Effects Assessment,15.0,"Other, All",http://www.cer-rec.gc.ca/esa-ees/2017ngtl/2017...,359,472.0,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-deve...,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...,-1,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...
27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1Y9J1,684515,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-2-2011,TERA Environmental Consultants,Operating,GC-119,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/7...,Appendix 8: Environmental Protection Plan,23.0,"Environment Protection Plan, All",http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln/n...,22,194.0,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigati...,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...,-1,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...
4165,13299,Table,Application for Trans Mountain Expansion Project,Trans Mountain Expansion,12/16/2013,Trans Mountain Pipeline ULC,Oil,B9-23 - V5C_TR_5C9_17of20_VEGETATION - A3S2L6,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3S2L6,2393491,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia, All",OH-001-2014,"BGC Engineering Inc., Waterline Resources Inc....",Approved,OC-064. OC-065,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Vegetation Technical Report,161.0,"Vegetation, Technology, All",http://www.cer-rec.gc.ca/esa-ees/tmx/tmx_table...,176,180.0,2.64,Yes,tmx,http://www.cer-rec.gc.ca/esa-ees/tmx.zip,False,tmx_table-d4-hope-to-burnaby-observed-plant-sp...,2393491_176_lattice-v_1.csv,TABLE D4 HOPE TO BURNABY OBSERVED PLANT SPECIE...,-1,TABLE D4 HOPE TO BURNABY OBSERVED PLANT SPECIE...
22455,31589,Table,"Application for Line 13 Transfer, Line 13 Reve...",Southern Lights,3/9/2007,Enbridge Southern Lights GP Inc.,Oil,B-2c Vol II - ESA - Section 3 - Consultation A...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0Y1T9,456043,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, Saskatchewan, Manitoba, All",OH-3-2007,TERA Environmental Consultants,Operating,XO-E242-03-2008,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,Section 3: Consultation,3.0,"Other, All",http://www.cer-rec.gc.ca/esa-ees/sthrnlghts/st...,15,24.0,0.28,Yes,sthrnlghts,http://www.cer-rec.gc.ca/esa-ees/sthrnlghts.zip,False,sthrnlghts_table-3-2-consultation-activities-w...,456043_15_lattice-v_1.csv,TABLE 32 CONSULTATION ACTIVI

In [7]:
# Explicitly ask for tensor allocation on CUDA device :0
classifier = pipeline("zero-shot-classification", device=0)

In [8]:
df_joined.iloc[100]['text']

'TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location KP Issue Timing Windows and or Regulatory Guidelines Mitigation Comments RARE PLANTS 2240 to 229 3640 to 3641 Iowa golden saxifrage NA Minimize surface disturbance by limiting grubbing to the trench line only If a level travel surface is needed for safety consider ramping over the travel lane with snow Restore contours following construction to maintain microtopography and hydrology Iowa golden saxifrage has been observed recolonizing disturbance at a variety of locations 070 to 100 820 to 821 1680 to 1681 2800 to 2801 3300 to 3301 3640 to 3641 3890 to 3891 7000 to 7001 western Jacobsladder NA Minimize surface disturbance by limiting grubbing to the trench line only If a level travel surface is needed for safety consider ramping over'

In [9]:
sequences = []

for x in range(len(df_joined)):
  sequences.append(df_joined.iloc[x]['text'])

In [10]:
len(sequences)

1000

In [11]:
sequences[0]

'TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION Map ID1 Title Proponent Description Construction Schedule2 Assessment Type Quantitative or Qualitative Physical Environment and Soils LSA Water Quality and Quantity RSA Air Quality RSA Acoustic Environment RSA Fish and Fish Habitat RSA Wetlands RSA Vegetation RSA Wildlife and Wildlife Habitat RSA HORU RSA Heritage Resources RSA TRLU RSA Community RSA 13307912 W6M to 6367913 W6M Trioil Resources Ltd Pipeline Quantitative 13307912 W6M to 13307912 W6M Trioil Resources Ltd Pipeline Quantitative 634876 W6M to 1034876 W6M Venturion Oil Limited Pipeline Quantitative 1234876 W6M to 634876 W6M Venturion Oil Limited Pipeline Quantitative 634876 W6M to 1234876 W6M Venturion Oil Limited Pipeline Quantitative 1234876 W6M to 634876 W6M Venturion Oil Limited Pipeline Quantitative 1034876 W6M'

In [12]:
candidate_labels = """Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples""".split(',')
hypothesis_template = "The data from this table is about {}."
candidate_labels

['Physical and meteorological environment',
 'Soil and soil productivity',
 'Vegetation',
 'Water quality and quantity',
 'Fish and fish habitat',
 'Wetlands',
 'Wildlife and wildlife habitat',
 'Species at Risk or Species of Special Status and related habitat',
 'Greenhouse gas (GHG) emissions and climate change',
 'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
 'Air emissions',
 'Acoustic environment',
 'Electromagnetism and Corona Discharge',
 'Human occupancy and resource use',
 'Heritage resources',
 'Navigation and navigation safety',
 'Traditional land and resource use',
 'Social and cultural well-being',
 'Human health and aesthetics',
 'Infrastructure and services',
 'Employment and economy',
 'Environmental Obligations',
 'Rights of Indigenous Peoples']

In [13]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [14]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1553 MB |    1553 MB |    1553 MB |       0 B  |\n|       from large pool |    1552 MB |    1552 MB |    1552 MB |       0 B  |\n|       from small pool |       1 MB |       1 MB |       1 MB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    1553 MB |    1553 MB |    1553 MB |       0 B  |\n|       from large pool |    1552 MB |    1552 MB |

In [15]:
from itertools import zip_longest
 
def grouper(iterable_obj, count, fillvalue=None):
    args = [iter(iterable_obj)] * count
    return zip_longest(*args, fillvalue=fillvalue)

output = [] 
 
# for e in range(epochs):
#     for images, labels in train_loader:   
#         if torch.cuda.is_available():
#             images, labels = images.cuda(), labels.cuda()   
#         # blablabla 

for sequence_batch in tqdm(grouper(sequences, 4, "")):
    print(sequence_batch)
    output.append(classifier(sequence_batch, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True))

0it [00:00, ?it/s]

('TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION Map ID1 Title Proponent Description Construction Schedule2 Assessment Type Quantitative or Qualitative Physical Environment and Soils LSA Water Quality and Quantity RSA Air Quality RSA Acoustic Environment RSA Fish and Fish Habitat RSA Wetlands RSA Vegetation RSA Wildlife and Wildlife Habitat RSA HORU RSA Heritage Resources RSA TRLU RSA Community RSA 13307912 W6M to 6367913 W6M Trioil Resources Ltd Pipeline Quantitative 13307912 W6M to 13307912 W6M Trioil Resources Ltd Pipeline Quantitative 634876 W6M to 1034876 W6M Venturion Oil Limited Pipeline Quantitative 1234876 W6M to 634876 W6M Venturion Oil Limited Pipeline Quantitative 634876 W6M to 1234876 W6M Venturion Oil Limited Pipeline Quantitative 1234876 W6M to 634876 W6M Venturion Oil Limited Pipeline Quantitative 1034876 W6M', 'TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Gui

1it [00:02,  2.05s/it]

('Table 5218 Major Proposed Approved and Recently Completed Projects in the SocioEconomic Study Area G8 Properties Station 44 Town Centre power centre big box stores hotel truck centre mixed density residential 5000 On hold NA2023', 'Table 20A4 Baseline Visual Value for the LAA and RAA Visual Value Rating Area in LAA ha Percent of Total LAA Area in RAA ha Percent of Total RAA Very low 734 12 4570 17 Low 1444 26 8254 30 Moderate 2290 41 10517 39 High 875 15 3553 13 Very high 353 6 412 2 Total 5657 100 27276 100', 'Table 216 Maximum Predicted GroundLevel Concentrations of CACs H2S Benzene and Mercaptans Project Alone Case Substance Averaging Period Maximum Predicted Ground Level Concentration gm3 Ambient Background Concentration gm3 Maximum Predicted Ground Level Concentration Including Background gm3 Regulatory Criteria gm3 CO 1hour 1160 03 1163 35000 CO 8hour 1078 02 1080 15000 H2S 1hour a 07 14 21 15 H2S 24hour 03 14 17 5 H2S Annual 20E03 05 05 Benzene 1hour a 102 102 30 Benzene 24hou

2it [00:03,  1.86s/it]

...

248it [06:33,  1.58s/it]

('Table 43 Criteria Air Contaminant Emissions from Permitted Point Sources in the Kitimat Terminal PEAA Source Criteria Air Contaminant Emissions tonnesyear Source SOXNOX CO TSPPM10PM25 TRS Area Sources Agriculture 00 00 00 00 00 00 Fuel Marketing 00 00 00 00 00 00 Landfill 00 00 00 13 05 01 Miscellaneous 00 00 03 33 24 21 Miscellaneous Burning 01 09 151 28 26 26 Other 00 00 00 162 15 06 Solvent Evaporation 00 00 00 00 00 00 Space Heating 89 405 4452 785 740 736 Total Area Sources 90 414 4607 1020 809 789 Mobile Sources Brake Lining 00 00 00 02 02 01 HeavyDuty Vehicles 07 358 123 12 12 11 LightDuty Vehicles 09 271 4771 04 04 04 Marine Vessels 2691 10266 711', 'TABLE 528 WATER LICENSES LOCATED IN THE LOCAL STUDY AREA IN MANITOBA KP Legal WPM Proximity to Proposed Route km Watercourse 10734 NW 21717 005 Souris River 11863 SW 1836 03 Thornhill Coulee 11966 SW 635 03 Deadhorse Creek 11966 635 03 Deadhorse Creek 11966 SE 635 03 Deadhorse Creek 11966 NW 33719 005 Deadhorse Creek 11966 635 03

249it [06:35,  1.58s/it]

('Table 611 ACIMSRecorded Occurrences of Plant Species and Ecological Communities of Management Concern in the Burton Creek CS Unit Addition RAA Plant FormCommunity Scientific Name Common Name ACIMSRanka AlbertaWildlifeActb COSEWICc SARAc Habitat Lichen Lecanorahypoptoides rimlichen S2 Not listed Not listed Not listed on wood rarely onbarkofDouglasfirh Lichen Lecidella patavina disk lichen S1S2 Not listed Not listed Not listed usually on calciumcontainingrocksg Lichen Leptogiumgelatinosum jellyskin lichen S2S3 Not listed Not listed Not listed usually on mossyrocks and soil rarelyontreesg Lichen Melanohaleasubelegantula camouflage lichen S3 Not listed Not listed Not listed common onbroadleaf trees in dryareasg Lichen Polysporinaarenacea cobblestone lichen S2 Not listed Not Listed Not Listed growsonrockf Moss Aulacomniumandrogynum little groove moss S2S3 Not listed Not listed Not listed organic soil rottinglogs mineral soil overrockf Moss Dicranum tauricum brokenleaf', 'TABLE 7242 POTENT

250it [06:37,  1.59s/it]


In [16]:
output[0]

[{'labels': ['Wildlife and wildlife habitat',
   'Water quality and quantity',
   'Fish and fish habitat',
   'Acoustic environment',
   'Physical and meteorological environment',
   'Vegetation',
   'Soil and soil productivity',
   'Wetlands',
   'Heritage resources',
   'Environmental Obligations',
   'Social and cultural well-being',
   'Air emissions',
   'Infrastructure and services',
   'Traditional land and resource use',
   'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
   'Species at Risk or Species of Special Status and related habitat',
   'Rights of Indigenous Peoples',
   'Greenhouse gas (GHG) emissions and climate change',
   'Employment and economy',
   'Human occupancy and resource use',
   'Navigation and navigation safety',
   'Electromagnetism and Corona Discharge',
   'Human health and aesthetics'],
  'scores': [0.9908032417297363,
   0.9892773032188416,
   0.9859839677810669,
   0.9473797678947449,
   0.9381691217422485,
   0.88502639532

In [17]:
# Since we have a list of lists for the outputs (because inference was done in batches), we need to flatten the lists together into one.
flatOutput = [item for elem in output for item in elem]

In [18]:
pickled_dataset = 'zero_shot_vcs_1k_for_labeling'

In [19]:
import pickle 
with open(f'{pickled_dataset}.pkl', 'wb') as f:
  pickle.dump(flatOutput, f)

In [20]:
import pickle
infile = open(f'{pickled_dataset}.pkl', 'rb')
zs_vcs_dict_list = pickle.load(infile)
infile.close()

In [1]:
zs_vcs_dict_list[0]

NameError: ignored

In [22]:
len(zs_vcs_dict_list)

1000

In [ ]:
for i in range(0, len(zs_vcs_dict_list)):
  zs_vcs_dict_list[0]['scores']

{'labels': ['Traditional land and resource use',
  'Water quality and quantity',
  'Social and cultural well-being',
  'Human occupancy and resource use',
  'Fish and fish habitat',
  'Heritage resources',
  'Navigation and navigation safety',
  'Vegetation',
  'Wetlands',
  'Physical and meteorological environment',
  'Soil and soil productivity',
  'Infrastructure',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Human health and aesthetics',
  'Electromagnetism and Corona Discharge',
  'Acoustic environment',
  'Air emissions',
  'Greenhouse gas (GHG) emissions and climate change'],
 'scores': [0.6877259612083435,
  0.649422287940979,
  0.4630372226238251,
  0.30576542019844055,
  0.08918734639883041,
  0.03485028073191643,
  0.03348839655518532,
  0.02901163510978222,
  0.017860960215330124,
  0.015356151387095451,
  0.012270933017134666,
  0.009136791341006756,
  0.008778736926615238,
  0.006375862751156092,
  0.004991511

In [ ]:
df_output = pd.DataFrame(output)
df_output

{'labels': ['Traditional land and resource use',
  'Water quality and quantity',
  'Social and cultural well-being',
  'Human occupancy and resource use',
  'Fish and fish habitat',
  'Heritage resources',
  'Navigation and navigation safety',
  'Vegetation',
  'Wetlands',
  'Physical and meteorological environment',
  'Soil and soil productivity',
  'Infrastructure',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Human health and aesthetics',
  'Electromagnetism and Corona Discharge',
  'Acoustic environment',
  'Air emissions',
  'Greenhouse gas (GHG) emissions and climate change'],
 'scores': [0.6877259612083435,
  0.649422287940979,
  0.4630372226238251,
  0.30576542019844055,
  0.08918734639883041,
  0.03485028073191643,
  0.03348839655518532,
  0.02901163510978222,
  0.017860960215330124,
  0.015356151387095451,
  0.012270933017134666,
  0.009136791341006756,
  0.008778736926615238,
  0.006375862751156092,
  0.004991511

In [ ]:
df_output[df_output.scores > 0.6]